Greatest common divisor/highest common factor:
    GCD := (λgmn. LEQ m n (g n m) (g m n)) (Y (λgxy. ISZERO y x (g y (MOD x y))))

[Details about algorithm](https://jwodder.freeshell.org/lambda.html)

In [1]:
import sys
import time

sys.path.append("../")
from calculus.term import *
from calculus.strategy import *
from calculus.advanced_terms import *

Less than:
LT := λab. NOT (LEQ b a)

Division — DIV a b evaluates to a pair of two numbers, a idiv b and a mod b:
DIV := Y (λgqab. LT a b (PAIR q a) (g (SUCC q) (SUB a b) b) ) 0

CDR p — get the cdr of pair p; also called SECOND, TAIL, or REST:
SECOND := CDR := λp. p FALSE

Modulus:
MOD := λab. CDR (DIV a b)

In [2]:
def lt_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, App(not_term(), multi_app_term(le_term(), b_, a_))))


def div_term():
    g, q, a, b = Var(), Var(), Var(), Var()
    g_, q_, a_, b_ = Atom(g), Atom(q), Atom(a), Atom(b)

    inner_right_term = multi_app_term(g_, App(succ_term(), q_), multi_app_term(substr_term(), a_, b_), b_)
    inner_lambda_term = Lambda(g, Lambda(q, Lambda(a, Lambda(b, multi_app_term(lt_term(), a_, b_, multi_app_term(pair_term(), q_, a_))))))

    return multi_app_term(y_term(), inner_lambda_term, inner_right_term, n_term(0))


def mod_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, App(second_term(), multi_app_term(div_term(), a_, b_))))

Greatest common divisor/highest common factor:
    GCD := (λgmn. LEQ m n (g n m) (g m n)) (Y (λgxy. ISZERO y x (g y (MOD x y))))

In [4]:
def gcd_term_v3():
    g, m, n, x, y = Var(), Var(), Var(), Var(), Var()
    g_, m_, n_, x_, y_ = Atom(g), Atom(m), Atom(n), Atom(x), Atom(y)

    left_inner_term = Lambda(g, Lambda(m, Lambda(n, multi_app_term(
        le_term(), m_, n_,
        multi_app_term(g_, n_, m_),
        multi_app_term(g_, m_, n_)
    ))))

    right_inner_term = Lambda(g, Lambda(x, Lambda(y, multi_app_term(
        iszero_term(), y_, x_,
        multi_app_term(g_, y_, multi_app_term(mod_term(), x_, y_))
    ))))
    right_inner_term = App(y_term(), right_inner_term)

    return App(left_inner_term, right_inner_term)

In [8]:
def test_gcd_v3(var_0=0, var_1=0, strategy=LeftmostOutermostStrategy()):
    start_time = time.time()
    term_ = multi_app_term(gcd_term_v3(), n_term(var_0), n_term(var_1))
    res_term, steps = term_.normalize_no_lim(strategy)

    print(f"Norm steps: {steps}")
    print(res_term)
    print(f"time normalization: {time.time() - start_time}s")

In [9]:
test_gcd_v3(0, 0, LeftmostOutermostStrategy())

Norm steps: 32
λq.λw.λe.e
time normalization: 0.037009239196777344s


In [10]:
test_gcd_v3(1, 0, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [11]:
test_gcd_v3(1, 0, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [12]:
test_gcd_v3(2, 2, LeftmostOutermostStrategy())

KeyboardInterrupt: 